In [1]:
import pyzed.sl as sl
import numpy as np
import cv2

In [2]:
zed = sl.Camera()

# 初始化ZED相机
init_params = sl.InitParameters()
init_params.camera_resolution = sl.RESOLUTION.HD1080
init_params.camera_fps = 30


status = zed.open(init_params)
if status != sl.ERROR_CODE.SUCCESS:
    print(f"相机打开失败：{status}")

runtime = sl.RuntimeParameters()

# 获取相机信息
camera_info = zed.get_camera_information()

In [4]:
camera_info.camera_configuration.resolution.width

1920

In [3]:


# Create a ZED camera object
zed = sl.Camera()

# Set configuration parameters
init_params = sl.InitParameters()
init_params.camera_resolution = sl.RESOLUTION.HD1080
init_params.camera_fps = 30


In [4]:
# Open the camera
err = zed.open(init_params)
if err != sl.ERROR_CODE.SUCCESS:
    exit()

In [5]:
# Get camera information (serial number)
zed_serial = zed.get_camera_information().serial_number
print("Hello! This is my serial number: {}".format(zed_serial))

Hello! This is my serial number: 36979282


In [17]:
# Capture 50 frames and stop
i = 0
image = sl.Mat()
runtime_parameters = sl.RuntimeParameters()
while i < 50:
    # Grab an image, a RuntimeParameters object must be given to grab()
    if zed.grab(runtime_parameters) == sl.ERROR_CODE.SUCCESS:
        # A new image is available if grab() returns ERROR_CODE.SUCCESS
        zed.retrieve_image(image, sl.VIEW.RIGHT) # Get the left image
        timestamp = zed.get_timestamp(sl.TIME_REFERENCE.IMAGE)  # Get the image timestamp
        print("Image resolution: {0} x {1} || Image timestamp: {2}\n".format(image.get_width(), image.get_height(), timestamp.get_milliseconds()))
        i = i + 1


Image resolution: 1920 x 1080 || Image timestamp: 1688452521384

Image resolution: 1920 x 1080 || Image timestamp: 1688452521418

Image resolution: 1920 x 1080 || Image timestamp: 1688452521451

Image resolution: 1920 x 1080 || Image timestamp: 1688452521485

Image resolution: 1920 x 1080 || Image timestamp: 1688452521518

Image resolution: 1920 x 1080 || Image timestamp: 1688452521552

Image resolution: 1920 x 1080 || Image timestamp: 1688452521585

Image resolution: 1920 x 1080 || Image timestamp: 1688452521619

Image resolution: 1920 x 1080 || Image timestamp: 1688452521652

Image resolution: 1920 x 1080 || Image timestamp: 1688452521685

Image resolution: 1920 x 1080 || Image timestamp: 1688452521719

Image resolution: 1920 x 1080 || Image timestamp: 1688452521752

Image resolution: 1920 x 1080 || Image timestamp: 1688452521786

Image resolution: 1920 x 1080 || Image timestamp: 1688452521819

Image resolution: 1920 x 1080 || Image timestamp: 1688452521853

Image resolution: 1920 x 

In [5]:
# Close the camera
zed.close()



In [18]:
img = image.numpy()

In [19]:
cv2.imshow('test', img[:, :, :3])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
########################################################################
#
# Copyright (c) 2022, STEREOLABS.
#
# All rights reserved.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS
# "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT
# LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR
# A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT
# OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL,
# SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT
# LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE,
# DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY
# THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
# (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
#
########################################################################

import sys
import pyzed.sl as sl
from signal import signal, SIGINT

cam = sl.Camera()

def handler(signal_received, frame):
    cam.disable_recording()
    cam.close()
    sys.exit(0)

signal(SIGINT, handler)

def main():
    if not sys.argv or len(sys.argv) != 2:
        print("Only the path of the output SVO file should be passed as argument.")
        exit(1)

    init = sl.InitParameters()
    init.camera_resolution = sl.RESOLUTION.HD720
    init.depth_mode = sl.DEPTH_MODE.NONE

    status = cam.open(init)
    if status != sl.ERROR_CODE.SUCCESS:
        print(repr(status))
        exit(1)

    path_output = sys.argv[1]
    recording_param = sl.RecordingParameters(path_output, sl.SVO_COMPRESSION_MODE.H264)
    err = cam.enable_recording(recording_param)
    if err != sl.ERROR_CODE.SUCCESS:
        print(repr(status))
        exit(1)

    runtime = sl.RuntimeParameters()
    print("SVO is Recording, use Ctrl-C to stop.")
    frames_recorded = 0

    while True:
        if cam.grab(runtime) == sl.ERROR_CODE.SUCCESS :
            frames_recorded += 1
            print("Frame count: " + str(frames_recorded), end="\r")

if __name__ == "__main__":
    main()

Only the path of the output SVO file should be passed as argument.
CAMERA NOT DETECTED
CAMERA NOT DETECTED
SVO is Recording, use Ctrl-C to stop.


SystemExit: 0

/home/aaron/miniconda3/envs/mp/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import pyzed.sl as sl
import cv2
import numpy as np

# 创建ZED相机对象并初始化
zed = sl.Camera()
init_params = sl.InitParameters()
err = zed.open(init_params)
if err != sl.ERROR_CODE.SUCCESS:
    print("Failed to open the camera")
    exit()

# 获取相机参数
camera_info = zed.get_camera_information()
rotation_matrix = np.array(camera_info.calibration_parameters.R).reshape(3, 3)
translation_vector = np.array(camera_info.calibration_parameters.T).reshape(3, 1)
extrinsic_matrix = np.concatenate((rotation_matrix, translation_vector), axis=1)

# 创建用于显示图像和外参矩阵的窗口
cv2.namedWindow("ZED", cv2.WINDOW_NORMAL)

# 捕获图像并实时显示
image = sl.Mat()
while True:
    # 捕获图像
    if zed.grab() == sl.ERROR_CODE.SUCCESS:
        zed.retrieve_image(image, sl.VIEW.LEFT)  # 获取左目图像

        # 将ZED图像转换为OpenCV格式
        image_ocv = image.get_data()

        # 在图像上显示外参矩阵
        for i in range(3):
            cv2.putText(image_ocv, f"R{i+1}:", (10, 20 + i * 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image_ocv, f"{extrinsic_matrix[i, 0]:.4f}", (80, 20 + i * 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image_ocv, f"{extrinsic_matrix[i, 1]:.4f}", (150, 20 + i * 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image_ocv, f"{extrinsic_matrix[i, 2]:.4f}", (220, 20 + i * 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image_ocv, f"{extrinsic_matrix[i, 3]:.4f}", (290, 20 + i * 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

        # 在窗口中显示图像
        cv2.imshow("ZED", image_ocv)

    # 检测键盘输入，按下ESC键退出循环
    key = cv2.waitKey(1)
    if key == 27:
        break

# 释放资源并关闭相机
zed.close()
cv2.destroyAllWindows()

AttributeError: 'pyzed.sl.CameraInformation' object has no attribute 'calibration_parameters'

In [2]:
import pyzed.sl as sl

def main():
    # 初始化ZED相機
    zed = sl.Camera()

    # 設置相機配置
    init_params = sl.InitParameters()
    init_params.camera_resolution = sl.RESOLUTION.HD1080
    init_params.camera_fps = 30

    # 開啟相機
    err = zed.open(init_params)
    if err != sl.ERROR_CODE.SUCCESS:
        print(f"相機無法開啟: {err}")
        exit()

    # 持續讀取相機數據
    runtime_params = sl.RuntimeParameters()
    mat = sl.Mat()
    pose = sl.Pose()

    while True:
        # 擷取一幀影像
        if zed.grab(runtime_params) == sl.ERROR_CODE.SUCCESS:
            # 獲取姿態信息
            zed.retrieve_image(mat, sl.VIEW.LEFT)
            zed.retrieve_measure(pose, sl.MEASURE.POSE)

            # 獲取旋轉和平移資訊
            rotation = pose.get_rotation_matrix()
            translation = pose.get_translation()

            # 輸出旋轉和平移資訊
            print("旋轉矩陣:")
            print(rotation)
            print("平移向量:")
            print(translation)

    # 關閉相機
    zed.close()

if __name__ == "__main__":
    main()

AttributeError: POSE

In [1]:
zed.close()

NameError: name 'zed' is not defined

In [3]:
import pyzed.sl as sl
import cv2

def main():
    # 初始化ZED相機
    zed = sl.Camera()

    # 設置相機配置
    init_params = sl.InitParameters()
    init_params.camera_resolution = sl.RESOLUTION.HD1080
    init_params.camera_fps = 30

    # 開啟相機
    err = zed.open(init_params)
    if err != sl.ERROR_CODE.SUCCESS:
        print(f"相機無法開啟: {err}")
        exit()

    # 獲取相機內部參數
    camera_info = zed.get_camera_information()

    # 創建OpenCV視窗
    cv2.namedWindow("ZED", cv2.WINDOW_NORMAL)

    # 持續讀取相機數據
    runtime_params = sl.RuntimeParameters()
    mat = sl.Mat()
    image_size = (camera_info.camera_resolution.width, camera_info.camera_resolution.height)

    while True:
        # 擷取一幀影像
        if zed.grab(runtime_params) == sl.ERROR_CODE.SUCCESS:
            # 獲取影像
            zed.retrieve_image(mat, sl.VIEW.LEFT)
            frame = mat.get_data()

            # 轉換影像格式
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # 顯示影像
            cv2.imshow("ZED", frame)

        # 檢查按鍵輸入
        key = cv2.waitKey(1)
        if key == ord("q"):
            break

    # 關閉相機
    zed.close()

    # 關閉OpenCV視窗
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

相機無法開啟: CAMERA NOT DETECTED


AttributeError: 'pyzed.sl.CameraInformation' object has no attribute 'camera_resolution'

: 